* 웹크롤러는 웹을 크롤링 하기 때문에 붙은 이름
* 그 핵심은 재귀
* URL에서 페이지를 가져오고, 그 페이지를 검사해 다른 URL 찾고..반복
* 타깃 서버의 부하를 줄일 방법을 강구해야 함

### 에릭 아이들의 페이지에서 시작해 케빈 베이컨의 페이지에 닿는 최소한의 클릭수 찾기

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import datetime
import random

In [2]:
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html, "html.parser")

# 너무 많아서 20개로 줄임
for link in bsObj.findAll("a", limit=20):
    if'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
http://baconbros.com/
#cite_note-1
#cite_note-actor-2
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)


* 예상대로 원치않는 것들이 많이 포함돼있음
* 사이드바, 푸터, 헤더 링크 등등

---
* 항목 페이지를 가리키는 링크는 다른 링크와 비교되는 특징이 있음
* 이 링크들은 id가 bodyContent인 div 안에 있음
* URL에는 세미콜론이 포함되어 있지 않음
* URL은 /wiki/로 시작함

In [3]:
# 너무 많아서 20개로 줄임
for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)(?!:).*$"), limit=20):
    if'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance


* 현 프로젝트에서는 모든 링크 목록을 가져오는 것은 별 쓸모 없음
* /wiki/<article_name> 형태인 위키백과 항목 URL을 받고
* 링크된 항목 URL 목록 전체를 반환하는 getLinks 함수
* 시작 항목에서 getLinks를 호출하고
* 반환된 리스트에서 무작위로 항목 링크 선택
* 다시 getLinks를 호출하는 작업
* 프로그램을 끝내거나 새 페이지에 항목 링크가 없을 때까지 반복

In [4]:
random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    return bsObj.find("div", {"id": "bodyContent"}).findAll("a", href=re.compile("^(/wiki/)(?!:).*$"))

links = getLinks("/wiki/Kevin_Bacon")
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

/wiki/Perry_v._Brown
/wiki/Twitter
/wiki/Naver_Corporation
/wiki/NTT_DoCoMo
/wiki/Seiko_Epson
/wiki/Toshiba
/wiki/Bandai_Namco_Entertainment
/wiki/Sora_Ltd.
/wiki/File:Sora_logo.svg
/wiki/Wikipedia:Non-free_use_rationale_guideline
/wiki/Wikipedia:Music_samples
/wiki/Wikipedia:Hatnote
/wiki/Tree_(descriptive_set_theory)
/wiki/Prefix_(computer_science)
/wiki/Formal_language_theory
/wiki/Von_Neumann%E2%80%93Bernays%E2%80%93G%C3%B6del_set_theory
/wiki/Formal_system
/wiki/Rule_of_inference
/wiki/Countable_set
/wiki/Russell%27s_paradox
/wiki/Forcing_(mathematics)
/wiki/Axiom
/wiki/Number_theory
/wiki/Rational_number
/wiki/Natural_numbers
/wiki/Dionysius_Exiguus
/wiki/Laurentius_Suslyga
/wiki/History
/wiki/Georges_Lefebvre


KeyboardInterrupt: 

In [6]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)(?!:).*$")):
        if'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 새 페이지를 발견
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks("")

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection
/wiki/Wikipedia:Requests_for_permissions
/wiki/Wikipedia:Requesting_copyright_permission
/wiki/Wikipedia:User_access_levels
/wiki/Wikipedia:Requests_for_adminship
/wiki/Wikipedia:Protection_policy#extended
/wiki/Wikipedia:Lists_of_protected_pages
/wiki/Wikipedia:Protection_policy
/wiki/Wikipedia:Perennial_proposals
/wiki/Wikipedia:Project_namespace#How-to_and_information_pages
/wiki/Wikipedia:Policies_and_guidelines
/wiki/Wikipedia:Shortcut
/wiki/Wikipedia:Keyboard_shortcuts
/wiki/Wikipedia:WikiProject_Kansas
/wiki/File:People_icon.svg
/wiki/Special:WhatLinksHere/File:People_icon.svg
/wiki/Help:What_links_here
/wiki/Wikipedia:Consensus#Levels_of_consensus
/wiki/Wikipedia:Conflict_of_interest
/wiki/Wikipedia:Protection_policy#move
/wiki/Wikipedia:WPPP
/wiki/Wikipedia:WikiProject
/wiki/Wikipedia:Wikimedia_sister_projects
/wiki/Help:Interwikimedia_links
/wiki/Help:Interlanguage_links
/wiki/

KeyboardInterrupt: 

In [7]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id="mw-content-text").findAll("p")[0])
        print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href'])
    except AttributeError:
        print("This page is missing something! No worries though!")
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)(?!:).*$")):
        if'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 새 페이지를 발견
                newPage = link.attrs['href']
                print("--\n"+ newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks("")

Main Page
<p>"<b><a href="/wiki/Habits_(Stay_High)" title="Habits (Stay High)">Habits (Stay High)</a></b>" is a song recorded by Swedish singer <a href="/wiki/Tove_Lo" title="Tove Lo">Tove Lo</a>, self-released as a single in March 2013, and re-released on 6 December 2013 by <a class="mw-redirect" href="/wiki/Universal_Music" title="Universal Music">Universal Music</a>. It appeared on her debut <a href="/wiki/Extended_play" title="Extended play">extended play</a>, <i><a href="/wiki/Truth_Serum_(EP)" title="Truth Serum (EP)">Truth Serum</a></i>, and was the <a href="/wiki/Lead_single" title="Lead single">lead single</a> from her debut studio album, <i><a href="/wiki/Queen_of_the_Clouds" title="Queen of the Clouds">Queen of the Clouds</a></i>. Written by Lo and its producers, Ludvig Söderberg and Jakob Jerlström, it is an <a href="/wiki/Electropop" title="Electropop">electropop</a> song featuring a <a href="/wiki/Minimal_music" title="Minimal music">minimal</a> and <a href="/wiki/Beat_(m

KeyboardInterrupt: 

In [8]:
scrapy startproject wikiSpider

SyntaxError: invalid syntax (<ipython-input-8-a79e88b6ec98>, line 1)